In [0]:
import pandas as pd
import numpy as np 
import keras 
from keras.layers import Dense, Embedding, LSTM, Input
import matplotlib.pyplot as plt 
from keras.models import Model
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
!nvidia-smi

Fri Apr 26 08:17:33 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

import os
os.chdir('drive/My Drive/Colab Notebooks/DataDays/second_round')

root@f72b65490fb7:/content# export PS1="root@f72b65490fb7-eb2a73197538452ba6e15572108c1fc3: "
root@f72b65490fb7-eb2a73197538452ba6e15572108c1fc3: umount -f /content/drive || umount /content/drive; pkill -9 -x drive
umount: /content/drive: no mount point specified.
umount: /content/drive: no mount point specified.
root@f72b65490fb7-eb2a73197538452ba6e15572108c1fc3: ( while `sleep 0.5`; do if [[ -d "/content/drive" && "$(ls -A /content/drive)" != "" ]]; then echo "google.colab.drive MOUNTED"; break; fi; done ) &
[1] 190
root@f72b65490fb7-eb2a73197538452ba6e15572108c1fc3: /opt/google/drive/drive --features=opendir_timeout_ms:15000,virtual_folders:true --inet_family=IPV4_ONLY --preferences=trusted_root_certs_file_path:/opt/google/drive/roots.pem,mount_point_path:/content/drive --console_auth
Could not read on-disk feature config: UNAVAILABLE_RESOURCE
[73624448] feature_config.cc:71:GetFeatureFlags Float type support in feature config has been removed
2019-04-26T08:18:30.292ZE [73624448] dr

In [0]:
import getpass
print(getpass.getpass())

··········



# Preprocess


In [0]:
pwd

'/content/drive/My Drive/Colab Notebooks/DataDays/second_round'

In [0]:
data = pd.read_csv('divar_posts_dataset.csv')

In [0]:
print(len(data.cat1.unique()))
data.cat1.unique()


6


array(['for-the-home', 'vehicles', 'personal', 'electronic-devices',
       'businesses', 'leisure-hobbies'], dtype=object)

In [0]:
print(len(data.cat2.unique()))
data.cat2.unique()


28


array(['furniture-and-home-decore', 'cars', 'baby-and-toys',
       'parts-accessories', 'utensils-and-appliances',
       'clothing-and-shoes', 'mobile-tablet',
       'childrens-clothing-and-shoe', 'game-consoles-and-video-games',
       'audio-video', 'building-and-garden', 'jewelry-and-watches',
       'equipments-and-machinery', 'bicycle', 'animals', nan, 'batch',
       'musical-instruments', 'health-beauty', 'motorcycles', 'computers',
       'sport-leisure', 'book-student-literature', 'utility',
       'travel-packages', 'hobby-collectibles', 'leisure-hobbies-toys',
       'phone'], dtype=object)

In [0]:
print(len(data.cat3.unique()))
data.cat3.unique()


67


array(['sofa-armchair', 'antiques-and-art', 'heavy', 'personal-toys', nan,
       'cookware-tableware', 'light', 'clothing', 'mobile-phones',
       'tv-projector', 'garden-and-patio', 'watches', 'offices',
       'kitchen', 'farm-animals', 'cat', 'video-dvdplayer',
       'shoes-belt-bag', 'industrial', 'tv-and-stereo-furniture', 'birds',
       'guitar-bass-amplifier', 'beds-bedroom', 'carpets',
       'mobile-tablet-accessories', 'fridge-and-freezer', 'lighting',
       'tables-and-chairs', 'strollers-and-accessories',
       'modem-and-network-equipment', 'jewelry', 'stereo-surround',
       'camera-camcoders', 'training', 'storage', 'stove-and-heating',
       'barbershop-and-beautysalon', 'dishwasher',
       'parts-and-accessories', 'cafe-and-restaurant', 'microwave-stove',
       'washer-dryer', 'educational', 'childrens-furniture',
       'piano-keyboard', 'desktops', 'shop-and-cash', 'laptops',
       'rhinestones', 'bathroom-wc-sauna', 'mp3-player',
       'textile-ornaments

In [0]:
data['cat1_2'] = data.cat1.astype(str).str.cat(data.cat2.astype(str), sep='|')
data['cat1_2_3'] = data.cat1_2.astype(str).str.cat(data.cat3.astype(str), sep='|')


In [0]:
train,test = train_test_split(data, random_state = 100)

In [0]:
labels = {j:i for i,j in enumerate(train.cat1_2_3.unique())}

In [0]:
X_train = train.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_train = train.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [0]:
X_test = test.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_test = test.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [0]:
MAX_NB_WORDS = 200000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(X_test)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

Found 200326 unique tokens.


In [0]:
# index_to_word = dict((i, w) for w, i in tokenizer.word_index.items())

In [0]:
MAX_SEQUENCE_LENGTH = 200 
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)


#Y_train = to_categorical(np.asarray(Y_train))
#Y_test = to_categorical(np.asarray(Y_test))


In [0]:
import codecs
from tqdm import *

In [0]:
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('model.txt', encoding='ISO-8859-1')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

0it [00:00, ?it/s]

loading word embeddings...


1228329it [00:51, 23816.97it/s]

found 856723 word vectors


In [0]:
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, 100))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


preparing embedding matrix...
number of null word embeddings: 190985


In [0]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
VALIDATION_SPLIT = 0.20
EMBEDDING_DIM = 100
N_CLASSES = 87
MAX_SEQUENCE_LENGTH = 200

filter_sizes = [2,3,4,5]
num_filters = 512
embedding_dim = 100
drop = 0.5
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            weights=[embedding_matrix],
                            trainable=False)
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)

# this creates a model that includes
model = Model(inputs=sequence_input, outputs=output)

checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

(?, 200, 100)
(?, 200, 100, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 100)     20000000    input_2[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 200, 100, 1)  0           embedding_2[0][0]                
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 199, 1, 512)  102912      reshape_2[0][0]                  
______________________________________________________________________________

In [0]:
model.fit(x_train, Y_train, validation_data=[x_test,Y_test],
          nb_epoch=10, batch_size=256)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


ValueError: ignored

In [0]:
model.fit(x_train, Y_train, validation_data=[x_test,Y_test],
          nb_epoch=10, batch_size=256)

In [0]:
model.save('./2dconvmodel4')

In [0]:
model.save('./2dconvmodel5')

In [0]:
from sklearn.metrics import roc_auc_score
output_test = model.predict(x_test)


In [0]:
from sklearn.metrics import classification_report
Y_test = np.argmax(Y_test,axis=1)
output_test = np.argmax(output_test,axis=1)
print(classification_report(Y_test, output_test))

In [0]:
tst = pd.read_csv('dataset.csv')

In [0]:
X_tst = tst.apply(lambda i : str(i.title) + ' ' + str(i.desc), axis=1)


In [0]:
texts_tst = X_tst.astype(str)


In [0]:
sequences_tst = tokenizer.texts_to_sequences(texts_tst)


In [0]:
MAX_SEQUENCE_LENGTH = 200

x_tst = pad_sequences(sequences_tst, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', x_tst.shape)

In [0]:
output_test = model.predict(x_tst)


In [0]:
pred=np.argmax(output_test,axis=1)

In [0]:
tst['cat1_2_3']=pred

In [0]:
merged = {}
for key, value in labels.items():
    merged[value] = key


In [0]:
tst['cat1_2_3'] = tst['cat1_2_3'].map(merged)

In [0]:
cat_tmp = tst['cat1_2_3'].str.split("|",n=1,expand=True)
tst['cat1'] = cat_tmp[0]
tst['cat2_3']=cat_tmp[1] 
cat2_tmp = tst['cat2_3'].str.split("|",n=1,expand=True)
tst['cat2'] = cat2_tmp[0]
tst['cat3']=cat2_tmp[1] 

In [0]:
tst = tst.drop(columns=['Unnamed: 0', 'archive_by_user', 'city', 'created_at', 'desc', 'id',
       'image_count', 'platform', 'price', 'title', 'cat1_2_3',
       'cat2_3',],axis=1)


In [0]:
tst.replace(['nan'], '', inplace=True)

In [0]:
tst.to_csv("conv2d_deep_em.csv")

In [0]:
model.save('./con2davalshodim_em')